# Building a Geomechanical Model from LOT's

This notebook demonstrates how to build a geomechanical model using Leak-Off Test (LOT) data with GeoSuite.

## Overview

Leak-Off Tests (LOTs) provide critical constraints for geomechanical models:
- Fracture gradient determination
- Minimum horizontal stress (Shmin) estimation
- Stress polygon constraints
- Mud weight window calculation

GeoSuite provides functions for:
- Overburden stress calculation from density logs
- Pore pressure prediction
- Stress polygon analysis
- Pressure and stress profiling

This notebook will show you how to:

1. Load well log data (density, sonic)
2. Calculate overburden stress
3. Estimate pore pressure
4. Use LOT data to constrain horizontal stresses
5. Build stress polygon and determine stress regime

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs
from geosuite.geomech import (
    calculate_overburden_stress,
    calculate_hydrostatic_pressure,
    calculate_pore_pressure_eaton,
    calculate_effective_stress,
    stress_polygon_limits,
    plot_stress_polygon,
    determine_stress_regime
)

print("GeoSuite imported successfully!")

## 1. Load Well Log Data

Load well log data including density (for overburden) and sonic (for pore pressure).

In [ ]:
# Load demo well log data
df = load_demo_well_logs()

print(f"Loaded {len(df):,} data points")
print(f"\nAvailable columns: {df.columns.tolist()}")

# Find depth, density, and sonic columns
depth_col = 'depth_m' if 'depth_m' in df.columns else 'DEPTH'
density_cols = [col for col in df.columns if 'RHOB' in col.upper() or 'DENSITY' in col.upper()]
sonic_cols = [col for col in df.columns if 'DT' in col.upper() or 'DTC' in col.upper() or 'SONIC' in col.upper()]

density_col = density_cols[0] if density_cols else None
sonic_col = sonic_cols[0] if sonic_cols else None

print(f"\nUsing depth column: {depth_col}")
print(f"Using density column: {density_col}")
print(f"Using sonic column: {sonic_col}")

df.head()

## 2. Calculate Overburden Stress

Use `calculate_overburden_stress()` to compute vertical stress (Sv) from density log. This function is Numba-accelerated for 20-50x speedup.

In [ ]:
# Calculate overburden stress from density log
if density_col:
    depth = df[depth_col].values
    rhob = df[density_col].values  # g/cc
    
    # Calculate Sv (overburden stress) in MPa
    sv = calculate_overburden_stress(depth, rhob)
    df['Sv'] = sv
    
    print(f"Overburden stress calculated")
    print(f"Sv range: {sv.min():.1f} to {sv.max():.1f} MPa")
    print(f"Sv gradient: {sv[-1]/depth[-1]*1000:.2f} kPa/m")
else:
    # Use typical gradient if no density log
    depth = df[depth_col].values if depth_col in df.columns else np.arange(len(df))
    sv = 0.023 * depth  # Typical gradient: 23 kPa/m = 0.023 MPa/m
    df['Sv'] = sv
    print("Using typical overburden gradient (23 kPa/m)")

## 3. Calculate Hydrostatic and Pore Pressure

Calculate hydrostatic pressure and estimate pore pressure using Eaton's method.

In [ ]:
# Calculate hydrostatic pressure
ph = calculate_hydrostatic_pressure(depth, rho_water=1.03)  # Brine density
df['Ph'] = ph

print(f"Hydrostatic pressure range: {ph.min():.1f} to {ph.max():.1f} MPa")
print(f"Hydrostatic gradient: {ph[-1]/depth[-1]*1000:.2f} kPa/m")

# Estimate pore pressure using Eaton's method (if sonic available)
if sonic_col:
    dt = df[sonic_col].values  # us/ft
    
    # Normal compaction trend (simplified - use actual trend in practice)
    dt_normal = 200 - 0.1 * depth  # Example trend
    dt_normal = np.clip(dt_normal, 40, 200)  # Reasonable bounds
    
    # Calculate pore pressure
    pp = calculate_pore_pressure_eaton(
        depth=depth,
        dt=dt,
        dt_normal=dt_normal,
        sv=sv,
        ph=ph,
        exponent=3.0  # Eaton exponent for sonic
    )
    df['Pp'] = pp
    
    print(f"\nPore pressure range: {pp.min():.1f} to {pp.max():.1f} MPa")
    print(f"Pore pressure gradient: {pp[-1]/depth[-1]*1000:.2f} kPa/m")
else:
    # Use hydrostatic if no sonic
    df['Pp'] = ph
    print("\nUsing hydrostatic for pore pressure (no sonic log)")

## 4. Use LOT Data to Constrain Shmin

LOT data provides direct measurements of minimum horizontal stress (Shmin). In practice, you would load actual LOT measurements.

In [ ]:
# Example LOT data (in practice, load from your database)
# LOT format: depth (m), LOT pressure (MPa)
lot_data = {
    'depth': [1000, 2000, 3000],  # Example depths
    'lot_pressure': [15.0, 30.0, 45.0]  # Example LOT pressures
}

print("Example LOT Data:")
print(f"Depths: {lot_data['depth']} m")
print(f"LOT Pressures: {lot_data['lot_pressure']} MPa")
print("\nNote: LOT pressure ≈ Shmin (minimum horizontal stress)")

# Interpolate Shmin from LOT data
shmin = np.interp(depth, lot_data['depth'], lot_data['lot_pressure'])
df['Shmin'] = shmin

print(f"\nShmin range: {shmin.min():.1f} to {shmin.max():.1f} MPa")
print(f"Shmin gradient: {shmin[-1]/depth[-1]*1000:.2f} kPa/m")

## 5. Stress Polygon Analysis

Use `stress_polygon_limits()` and `plot_stress_polygon()` to determine allowable stress regimes and constrain SHmax.

In [ ]:
# Calculate stress polygon limits at a specific depth
# Example: analyze at 2000 m depth
analysis_depth = 2000.0
idx = np.argmin(np.abs(depth - analysis_depth))

sv_val = sv[idx]
pp_val = df['Pp'].values[idx]
shmin_val = shmin[idx]

print(f"Stress Analysis at {analysis_depth:.0f} m depth:")
print(f"  Sv (overburden): {sv_val:.1f} MPa")
print(f"  Pp (pore pressure): {pp_val:.1f} MPa")
print(f"  Shmin (from LOT): {shmin_val:.1f} MPa")

# Calculate stress polygon limits
limits = stress_polygon_limits(
    sv=sv_val,
    pp=pp_val,
    shmin=shmin_val,
    mu=0.6,  # Coefficient of friction
    cohesion=0.0
)

print(f"\nStress Polygon Limits (SHmax range):")
print(f"  Normal faulting: {limits['normal'][0]:.1f} - {limits['normal'][1]:.1f} MPa")
print(f"  Strike-slip: {limits['strike_slip'][0]:.1f} - {limits['strike_slip'][1]:.1f} MPa")
if limits['reverse'][1] is not None:
    print(f"  Reverse faulting: {limits['reverse'][0]:.1f} - {limits['reverse'][1]:.1f} MPa")

# Determine most likely regime
if shmin_val < sv_val:
    print(f"\nMost likely regime: Normal faulting (Shmin < Sv)")
    shmax_range = limits['normal']
elif shmin_val > sv_val:
    print(f"\nMost likely regime: Reverse faulting (Shmin > Sv)")
    shmax_range = limits['reverse'] if limits['reverse'][1] else limits['strike_slip']
else:
    print(f"\nMost likely regime: Strike-slip (Shmin ≈ Sv)")
    shmax_range = limits['strike_slip']

## 6. Visualize Stress Polygon

Create a stress polygon plot showing allowable stress regimes with depth.

In [ ]:
# Create stress polygon plot
fig = plot_stress_polygon(
    depths=depth,
    sv=sv,
    pp=df['Pp'].values,
    shmin=shmin,
    mu=0.6,
    cohesion=0.0,
    title='Stress Polygon - Geomechanical Model'
)

plt.show()

## 7. Summary

This notebook demonstrated:

-  Calculating overburden stress with `calculate_overburden_stress()` (Numba-accelerated)
-  Estimating pore pressure with `calculate_pore_pressure_eaton()`
-  Using LOT data to constrain Shmin
-  Stress polygon analysis with `stress_polygon_limits()`
-  Visualizing stress regimes with `plot_stress_polygon()`

### Key Functions Used

- `calculate_overburden_stress()`: Compute Sv from density log
- `calculate_hydrostatic_pressure()`: Calculate Ph
- `calculate_pore_pressure_eaton()`: Estimate Pp from sonic
- `stress_polygon_limits()`: Determine SHmax constraints
- `plot_stress_polygon()`: Visualize stress regimes

### Next Steps

- Load actual LOT data from your database
- Use multiple LOTs to constrain Shmin profile
- Incorporate wellbore failure observations (breakouts, tensile fractures)
- Calculate mud weight windows for drilling
- Export stress model for well planning